In [ ]:
# 查看当前挂载的数据集目录
!ls /home/mw/input/

In [ ]:
# 查看个人持久化工作区文件
!ls /home/mw/work/

In [ ]:
# 查看当前kernel下的package
!pip list --format=columns

In [ ]:
# 显示cell运行时长
%load_ext klab-autotime

In [20]:
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
import pandas as pd
import gc
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.metrics import confusion_matrix
%matplotlib inline
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
import lightgbm as lgb
from sklearn.metrics import fbeta_score
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import cross_val_score # 交叉验证

import pandas  as pd

In [3]:
# lgb_oof = pd.read_csv('lgb_oof.csv')
# xgb_oof = pd.read_csv('xgb_oof.csv')
# cbt_oof = pd.read_csv('cbt_oof.csv')
lgb_oof = pd.read_csv('lgb_oof_feature_engineering.csv')
xgb_oof = pd.read_csv('xgb_oof_feature_engineering.csv')
cbt_oof = pd.read_csv('cbt_oof_feature_engineering.csv')

In [4]:
lgb_oof.head()

,user_id,y,pred
0,A00006,0,0.411000
1,A00017,1,0.417198
2,A00064,0,0.170268
3,A00081,0,0.418154
4,A00086,0,0.178812


In [5]:
oof_t =pd.merge(lgb_oof[['user_id','pred','y']],xgb_oof[['user_id','pred']],on='user_id',how='inner',suffixes = ['_lgb','_xgb'])
oof =pd.merge(oof_t,cbt_oof[['user_id','pred']],on='user_id',how='inner')

oof.head()

,user_id,pred_lgb,y,pred_xgb,pred
0,A00006,0.411000,0,0.403242,0.422633
1,A00017,0.417198,1,0.389520,0.372451
2,A00064,0.170268,0,0.055312,0.061393
3,A00081,0.418154,0,0.406562,0.462052
4,A00086,0.178812,0,0.043656,0.053881


In [6]:
feature = ['pred_lgb'	,	'pred_xgb',	'pred']
ycol = 'y'

In [7]:


import numpy as np
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split



log_reg = LogisticRegression(multi_class='ovr', solver='sag')

X_train, X_test, Y_train, Y_test = train_test_split(oof[feature], oof[ycol], test_size=0.2)
# 需要验证的超参数，tol收敛的阈值，C正则化惩罚因子的倒数， C越大，惩罚项越弱
param_grid = {
    'tol': [1e-4, 1e-3, 1e-2],
    'C': [0.4, 0.6, 0.8]
}

# 网格搜索（计算每种可能的参数组合后的分数），采用3折交叉验证（数据分为三份，分别作为验证集，得出分数）
# grid_search = GridSearchCV(log_reg, param_grid, cv=3,scoring='roc_auc')
# grid_search = GridSearchCV(log_reg, param_grid, cv=3,scoring='f1') 
# grid_search = GridSearchCV(log_reg, param_grid, cv=3,scoring='precision')  
# grid_search = GridSearchCV(log_reg, param_grid, cv=3,scoring='accuracy')
grid_search = GridSearchCV(log_reg, param_grid, cv=3,scoring='recall')
grid_search.fit(X_train, Y_train)

print(grid_search.score(X_test, Y_test))  # 分数
print(grid_search.best_params_)  # 最好的超参数
# print(grid_search.best_estimator_)  # 逻辑回归的参数设置（交叉验证后最好的参数）


0.22105263157894736
{'C': 0.8, 'tol': 0.0001}


In [8]:
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'brier_score_loss',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'v_measure_score']

In [32]:
lr = LogisticRegression(C=1000.0, random_state=0)
lr.fit(X_train, Y_train)
Y_pre = lr.predict_proba(X_test)[:,1] # 查看第一个测试样本属于各个类别的概率
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
Threshold = 0.35


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [33]:
recall_score( Y_test, Y_pre >Threshold ,average='macro')


0.6848189838758747

In [34]:
accuracy_score(Y_test, Y_pre >Threshold)

0.8049886621315193

In [35]:
f1_score(Y_test, Y_pre >Threshold,average='macro')


0.6947753026010817

In [36]:
precision_score(Y_test, Y_pre >Threshold,average='macro')

0.7083333333333334

In [37]:
def get_roc_auc(y_test,y_score):
    # y_score =clf.predict_proba(test_vecs)[:,1]
    auc_s = roc_auc_score(y_test,y_score)   
    # print(y_predict)
    # print(y_score)
#     print(cm)
    print(auc_s)
    # Compute ROC curve and ROC area for each class
    fpr,tpr,threshold = roc_curve(y_test, y_score) ###计算真正率和假正率
    roc_auc = auc(fpr,tpr) ###计算auc的值
    
    plt.figure()
    lw = 2
    plt.figure(figsize=(10,10))
    plt.plot(fpr, tpr, color='darkorange',
            lw=lw, label='ROC curve (area = %0.2f)' % roc_auc) ###假正率为横坐标，真正率为纵坐标做曲线
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

In [38]:
get_roc_auc(Y_test, Y_pre)

0.7940066930331608


<Figure size 432x288 with 0 Axes>

<Figure size 720x720 with 1 Axes>

In [39]:


from sklearn.metrics import precision_recall_curve
# from sklearn.utils.fixes import signature
plt.figure("P-R Curve")
plt.title('Precision/Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
precision, recall, thresholds = precision_recall_curve(Y_test, Y_pre )
#print(precision)
#print(recall)
#print(thresholds)
plt.plot(recall,precision)
plt.show()

<Figure size 432x288 with 1 Axes>

In [40]:
fpr,tpr,thresholds= roc_curve(Y_test, Y_pre )
print ('KS:',max(tpr-fpr))

KS: 0.501886218436264


In [41]:
max_f1 = 0
Threshold = 0
for i in range(200,400):
    f1 = f1_score(Y_test, Y_pre >i * 1e-3,average='macro')
    # f1 = f1_score(df_oof['y'].astype('int64'), (df_oof['pred'] >=i * 1e-3).astype('int64'),average='weighted')
    if max_f1 < f1:
        max_f1 = f1
        Threshold =  i

max_f1 ,Threshold * 1e-3

(0.6947753026010817, 0.34900000000000003)